In [1]:
import numpy
import pandas
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
# This quickly calculates the accuracy:
def calculate_accuracy(df):
    acc = {}
    for key in ['neut', 'npi', 'cpi', 'prot']:
        accuracy = numpy.mean(df['true_{}'.format(key)] == df['pred_{}'.format(key)])
        acc[key] = accuracy
#         print("{0}: {1:.3}%".format(key, 100*accuracy))
    return acc


In [3]:
# Fetch the 2D and 3D dataframes:
files_3d = [ "/Users/corey.adams/data/wire_pixel_analysis_files/3D_baseline/val_event_id_{}_of_8_out.pd".format(i) for i in range(1,9)]
df_list = [pandas.read_pickle(_file) for _file in files_3d]
#concatenate them together
df3d = pandas.concat(df_list, ignore_index=True)

files_2d = [ "/Users/corey.adams/data/wire_pixel_analysis_files/2D_baseline/val_event_id_{}_of_8_out.pd".format(i) for i in range(1,9)]
df_list = [pandas.read_pickle(_file) for _file in files_2d]
#concatenate them together
df2d = pandas.concat(df_list, ignore_index=True)



In [4]:
# nc_2d_good_events = df2d.query("pred_neut0 < 0.32 | pred_neut0 > 0.34")
df2d = df2d.iloc[nc_2d_good_events.index]
df3d = df3d.iloc[nc_2d_good_events.index]

NameError: name 'nc_2d_good_events' is not defined

In [ ]:
accuracy_df = pandas.DataFrame(index=['all', 'nueCC', 'numuCC', 'NC'], 
                               columns=["neut2D", "neut3D", 
                                        "npi2D",  "npi3D",
                                        "cpi2D",  "cpi3D",
                                        "prot2D", "prot3D"]
                              )
for dim in ['2D', '3D']:
    df = df2d if dim == '2D' else df3d
    accs = calculate_accuracy(df)
    for i, selection in enumerate(['nueCC', 'numuCC', 'NC']):
        sub_accs = calculate_accuracy(df.query("true_neut=={}".format(i)))
        for key in ['neut', 'npi', 'cpi', 'prot']:
            if i == 0:
                accuracy_df.loc['all'][key+dim] = "{0:.1f}%".format(100*accs[key])
            accuracy_df.loc[selection][key+dim] = "{0:.1f}%".format(100*sub_accs[key])


In [ ]:
accuracy_df

In [ ]:
def normalize(_df, target=1e16):
    for category in [0,1,2]:
        sub_df = _df.query('true_neut=={}'.format(category))
        weight = target / numpy.sum(sub_df['pot'])
        
        _df.loc[_df["true_neut"] == category, "weight"] = weight



In [ ]:
normalize(df2d)
normalize(df3d)

In [ ]:
print(numpy.sum(df2d['pot']))
print(numpy.sum(df3d['pot']))

In [ ]:
print(len(df))

In [ ]:
# This function creates a histogram, based on true energy, that takes a dataframe as input
def plot_events(df, categories, bins = numpy.arange(0,3.0, 0.2), title=""):

    # categories is a list of dicts.  Each dictionary consists of
    # a set of true selections passed to df.query
    # a label name for the legend
    # a color
    
    
    fig = plt.figure(figsize=(16,9))

    bottom = numpy.zeros(len(bins) -1 )

    for d in categories:
        sub_df = df.query(d['selection'])
        energies = sub_df['energy']
        weights  = sub_df['weight']
        
        
        # Use the passed bins to turn the energies into a histogram:
        values, bin_edges = numpy.histogram(energies, bins=bins, weights=weights)
    
        bin_centers = 0.5*(bin_edges[1:] + bin_edges[:-1])
        bin_widths  = (bin_edges[1:] - bin_edges[:-1])

        plt.bar(bin_centers, values, 
                align='center', 
                width=bin_widths, 
                bottom=bottom,
                color=d['color'],
                label=d['label']
               )
        bottom += values


    ax = plt.gca()
    ax.set_axisbelow(True)
    plt.grid(True)
    plt.legend(fontsize=25)
    
    plt.ylabel("Events / 1e20 POT", fontsize=20)
    plt.xlabel("True Energy [GeV]", fontsize=20)

    plt.title(title, fontsize=25)

    for tick in ax.xaxis.get_major_ticks():
        tick.label.set_fontsize(20)
    for tick in ax.yaxis.get_major_ticks():
        tick.label.set_fontsize(20)
    
    plt.show()
        

In [ ]:
categories = [
    {
        'selection' : "true_neut == 0",
        'color'     : "blue",
        'label'     : r"$\nu_e CC$"
    },
    {
        'selection' : "true_neut == 1",
        'color'     : "green",
        'label'     : r"$\nu_{\mu} CC$"
    },
    {
        'selection' : "true_neut == 2 ",
        'color'     : "red",
        'label'     : r"$NC$"
    }
]
categories_pi0 = [
    {
        'selection' : "true_neut == 0 & true_npi==0",
        'color'     : "blue",
        'label'     : r"$\nu_e CC Other$"
    },
    {
        'selection' : "true_neut == 0 & true_npi==1",
        'color'     : "lightblue",
        'label'     : r"$\nu_e CC$ w/ $\pi^0$"
    },
    {
        'selection' : "true_neut == 1 & true_npi==0",
        'color'     : "green",
        'label'     : r"$\nu_{\mu} CC$"
    },
    {
        'selection' : "true_neut == 1 & true_npi==1",
        'color'     : "lightgreen",
        'label'     : r"$\nu_{\mu} CC$ w/ $\pi^0$"
    },
    {
        'selection' : "true_neut == 2 & true_npi==0",
        'color'     : "pink",
        'label'     : r"$NC$ Other"
    },
    {
        'selection' : "true_neut == 2 & true_npi==1",
        'color'     : "red",
        'label'     : r"$NC$ w/ $\pi^0$"
    }
]

In [ ]:

plot_events(df2d, categories=categories_pi0, title="2D")
plot_events(df3d, categories=categories_pi0, title="3D")

In [ ]:
# selection="pred_neut == 0"
selection="pred_neut0>0.9"
plot_events(df2d.query(selection), categories=categories_pi0, title="2D")
plot_events(df3d.query(selection), categories=categories_pi0, title="3D")

In [ ]:
def get_metrics(df, true_selection, reco_selection):
    # This function computes the efficiency and purity for a reco selection on top of a truth selection.
    
    true_df = df.query(true_selection)
    selected_df = df.query(reco_selection)
    
    background = df.query(reco_selection)
    n_selected = numpy.sum(selected_df['weight'])
    n_true = numpy.sum(true_df['weight'])
    
    n_correct_selected = numpy.sum(selected_df.query(true_selection)['weight'])
    n_incorrect_selected = n_selected - n_correct_selected

    purity = n_correct_selected / n_selected
    efficiency = n_correct_selected / n_true
    
    return {
        "efficiency" : efficiency, 
        "purity" : purity,
    }


In [ ]:
print(get_metrics(df2d, true_selection="true_neut==0", reco_selection="pred_neut0 >= 0.95"))
print(get_metrics(df3d, true_selection="true_neut==0", reco_selection="pred_neut0 >= 0.95"))

In [ ]:
def make_roc_curve(df, cut_string, coordinates, true_selection):
    # This function populates the efficiency and accuracy values by varying
    # the coordinates fed in.  
    
    efficiency = numpy.zeros(len(coordinates))
    purity = numpy.zeros(len(coordinates))


    
    for i, coor in enumerate(coordinates):
        cut = cut_string.format(coor)
        res = get_metrics(df, true_selection, cut)
        efficiency[i] = res['efficiency']
        purity[i] = res['purity']        

    
    # prune off any NaN indexes
    valid = numpy.logical_not(numpy.isnan(purity))
            
    return efficiency[valid], purity[valid]

In [ ]:
e2d_nue, p2d_nue = make_roc_curve(df2d, 
               cut_string="pred_neut0>{}", 
               coordinates = numpy.arange(0,1.0,0.001), 
               true_selection="true_neut==0")

e3d_nue, p3d_nue = make_roc_curve(df3d, 
               cut_string="pred_neut0>{}", 
               coordinates = numpy.arange(0,1.0,0.001), 
               true_selection="true_neut==0")

In [ ]:
e2d_numu, p2d_numu = make_roc_curve(df2d, 
               cut_string="pred_neut1>{}", 
               coordinates = numpy.arange(0,1.0,0.001), 
               true_selection="true_neut==1")

e3d_numu, p3d_numu = make_roc_curve(df3d, 
               cut_string="pred_neut1>{}", 
               coordinates = numpy.arange(0,1.0,0.001), 
               true_selection="true_neut==1")

In [ ]:
e2d_NC, p2d_NC = make_roc_curve(df2d, 
               cut_string="pred_neut2>{}", 
               coordinates = numpy.arange(0,1.0,0.001), 
               true_selection="true_neut==2")

e3d_NC, p3d_NC = make_roc_curve(df3d, 
               cut_string="pred_neut2>{}", 
               coordinates = numpy.arange(0,1.0,0.001), 
               true_selection="true_neut==2")

In [ ]:
def plot_roc_curve(curves, title=""):
    
    fig = plt.figure(figsize=(16, 16))

    for e, p, label in curves:

        plt.plot(e, p, label=label, lw=3)

        plt.xlabel("Purity", fontsize=25)
        plt.ylabel("Efficiency", fontsize=25)
        plt.grid(True)

        
    plt.legend(fontsize=25)
    
    plt.title(title, fontsize=35)
    plt.ylabel("Purity", fontsize=30)
    plt.xlabel("Efficiency", fontsize=30)

    ax = plt.gca()
    for tick in ax.xaxis.get_major_ticks():
        tick.label.set_fontsize(20)
    for tick in ax.yaxis.get_major_ticks():
        tick.label.set_fontsize(20)
    
    plt.ylim([0,1])
    plt.xlim([0,1])


    plt.show()

In [ ]:
plot_roc_curve(curves=[
                          [e2d_nue, p2d_nue, "2d"],
                          [e3d_nue, p3d_nue, "3d"]
                      ],
               title="Nue CC"
    )

In [ ]:
plot_roc_curve(curves=[
                          [e2d_numu, p2d_numu, "2d"],
                          [e3d_numu, p3d_numu, "3d"]
                      ],
               title="Numu CC"
    )

In [ ]:
plot_roc_curve(curves=[
                          [e2d_NC, p2d_NC, "2d"],
                          [e3d_NC, p3d_NC, "3d"]
                      ],
               title="NC"
    )

In [ ]:
def print_stats(df, selection):
    print("Total number of events selected by this selection: ", numpy.sum(df.query(selection)['weight']))
    print("Total number of nue events selected by this selection: ", numpy.sum(df.query(selection).query("true_neut==0")['weight']))
    print("Total number of numu events selected by this selection: ", numpy.sum(df.query(selection).query("true_neut==1")['weight']))
    print("Total number of nc events selected by this selection: ", numpy.sum(df.query(selection).query("true_neut==2")['weight']))





In [ ]:
selection= 'pred_neut0>0.5'
# selection= 'pred_neut=50 | pred_neut==1 | pred_neut==2'

print_stats(df2d, selection)
print()
print_stats(df3d, selection)




In [ ]:
0.48 / (0.48 + 0.45 + 3.69)

In [ ]:
0.48 / (0.48 + 1.66 + 1.42)

In [ ]:
bins = numpy.arange(0,1.001, 0.01)

bin_centers = 0.5*(bins[1:] + bins[:-1])
bin_widths  = bins[1:] - bins[:-1]


labels = 'nue', 'numu', 'nc'
bottom = numpy.zeros(len(bins) - 1)

fig = plt.figure(figsize=(16,9))

for i, selection in enumerate(['true_neut==0', 'true_neut==1', 'true_neut==2']):
    vals, be = numpy.histogram(df2d.query(selection)['pred_neut0'], bins=bins)
    plt.bar(bin_centers, vals, bottom=bottom, width=bin_widths, label=labels[i])
    bottom += vals
    
plt.xlabel("Nue Prediction Score")
plt.yscale('log')
plt.legend(fontsize=25)
plt.title('2D')
plt.show()


In [ ]:
bins = numpy.arange(0,1.001, 0.01)

bin_centers = 0.5*(bins[1:] + bins[:-1])
bin_widths  = bins[1:] - bins[:-1]


labels = 'nue', 'numu', 'nc'
bottom = numpy.zeros(len(bins) - 1)

fig = plt.figure(figsize=(16,9))

for i, selection in enumerate(['true_neut==0', 'true_neut==1', 'true_neut==2']):
    vals, be = numpy.histogram(df3d.query(selection)['pred_neut0'], bins=bins)
    plt.bar(bin_centers, vals, bottom=bottom, width=bin_widths, label=labels[i])
    bottom += vals
    
plt.xlabel("Nue Prediction Score")
plt.legend(fontsize=25)
plt.yscale('log')
plt.title('3D')
plt.show()

In [ ]:
numpy.sum(df3d.query('true_neut == 0 & pred_neut0 < 0.5')['weight'])

In [ ]:
print(get_metrics(df2d_corrected, true_selection="true_neut==0", reco_selection="pred_neut0 >= 0.95"))
print(get_metrics(df3d_corrected, true_selection="true_neut==0", reco_selection="pred_neut0 >= 0.95"))

In [ ]:
df3d_corrected